# Chapter.5 모델 평가와 성능 향상

In [3]:
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [12]:
X, y = make_blobs(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

logreg = LogisticRegression().fit(X_train, y_train)
print('테스트 세트 점수 : {:.2f}'.format(logreg.score(X_test, y_test)))

테스트 세트 점수 : 0.88


In [7]:
X.shape

(100, 2)

In [8]:
y.shape

(100,)